# Grounding Guardrail 

### Fact-Checking

The goal of the self-check fact-checking output rail is to ensure that the answer to a RAG (Retrieval Augmented Generation) query is grounded in the provided evidence extracted from the knowledge base (KB).

NeMo Guardrails uses the concept of **relevant chunks** (which are stored in the `$relevant_chunks` context variable) as the evidence against which fact-checking should be performed. The relevant chunks can be extracted automatically, if the built-in knowledge base support is used, or provided directly alongside the query (see the [Getting Started Guide example](../getting_started/7_rag)).

**IMPORTANT**: The performance of this rail is strongly dependent on the capability of the LLM to follow the instructions in the `self_check_facts` prompt.

In [15]:
import sys
import os

module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [16]:
import json
import boto3
from utils import bedrock, print_ww
import pandas as pd

# This helper function encompasses the process of initializing NeMo Guardrails and generating Rails based on a specified configuration.
from nemoguardrails import LLMRails, RailsConfig

# BedrockModels is a "Singleton" class which initializes the necessary models for the notebook.
from models import BedrockModels

# This creates a pywidget chat window for testing
from models import ChatComponent

## Initalize bedrock client

In [28]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Define function to create a LLMRails instance with Claudev2 as LLM and a certain rail configuration


In [29]:
# This helper function encapsulates the necessary steps to bootstrap
# NeMo Guardrails and returns Rails based on a given configuration.
def bootstrap_bedrock_nemo_guardrails(rail_config_path: str) -> LLMRails:

    #1. initialize rails config
    config = RailsConfig.from_path(f"NeMo/rails/{rail_config_path}/config")

    # initialize bedrock models
    # you can pass model id as string or use the default model id 'anthropic.claude-v2'
    bedrock_models = BedrockModels
    bedrock_models.init_bedrock_client(boto3_bedrock)
    bedrock_models.init_llm('anthropic.claude-v2')

    # 2. bootstraps NeMo Guardrails with the necessary resources
    app = LLMRails(config=config, llm=bedrock_models.llm,  verbose=True)
    
    return app


## Test prompt without Grounding Guardrail which creates hallucination

In [19]:
example_prompt = "What was the unemployment rate for teenagers in the US in 2023?"

In [20]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v2",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )


In [17]:
response = textgen_llm(example_prompt)

print_ww(response)

 I'm afraid I don't have enough information to provide the exact unemployment rate for teenagers in
the US in 2023, since we are still in 2022. The unemployment rate can fluctuate year to year.
However, here is some helpful context:

- In 2022, the unemployment rate for teenagers aged 16-19 in the US was around 10.4%, according to
the Bureau of Labor Statistics. This was a decrease from peak pandemic levels of over 20% in 2020.

- The teen unemployment rate tends to be significantly higher than the national average, which was
around 3.7% in 2022. This is often attributed to teens having less work experience and competing
with older workers.

- Prior to the pandemic in 2019, the unemployment rate for 16-19 year olds was around 12.6%.

- The youth unemployment rate is influenced by the overall economy, minimum wage levels,
availability of entry-level jobs, and enrollment in school or training programs.

- Most economists expect the job market to remain relatively strong in 2023 compared 

## Test prompt with Grounding Guardrail

In [33]:
# Bootstrap Guardrails with grounding configuration
grounding_llm = bootstrap_bedrock_nemo_guardrails('grounding')


Human conversations vector store index loaded from disk.

Assistant conversations vector store index loaded from disk.

NeMo Conversations Flows vector store index loaded from disk.

KnowledgeBase vector store index loaded from disk.


In [31]:
bot_message = grounding_llm.generate(prompt="In which year did the DARPA announce a multi-year investment?")
print(bot_message)

Parameter temperature does not exist for Bedrock
Parameter temperature does not exist for Bedrock


Based on the context provided, it seems DARPA announced a multi-year investment of over $2 billion for AI research and development in 2018, as part of its AI Next campaign:
"In 2018 the Defense Advanced Research Project Agency (DARPA) announced a multi-year investment of more than $2 billion in new and existing programs and called it the AI Next campaign."


In [37]:
# asynchronous invocation with history
history.append(bot_message)
history.append(
    {"role": "user", "content": "What was the unemployment rate for teenagers?"}
)
bot_message = await grounding_llm.generate_async(messages=history)
print(bot_message['content'])

Parameter temperature does not exist for Bedrock
Parameter temperature does not exist for Bedrock


Unfortunately I do not have sufficient information to provide an exact unemployment rate for teenagers. However, here is some general information about teenage unemployment:
- Teenagers typically have higher unemployment rates compared to the overall population. This is often due to lack of work experience and competition from older, more skilled workers.
- In the United States, the teenage unemployment rate fluctuates over time but tends to be around 2-3 times higher than the national unemployment rate.
- During economic downturns and recessions, teenage unemployment can spike significantly higher due to businesses cutting back hiring.
- The unemployment rate for teenagers varies based on factors like age, education level, and demographics. For example, older teens tend to have lower unemployment than younger teens.
- Government agencies like the Bureau of Labor Statistics track and report on teenage unemployment rates monthly and annually. However, without specifics on location and t

In [ ]:
## Experiment with the Chat Interface and Review Sample Questions Ahead
Please feel free to interact with the chat below. Once completed, proceed to the next block to examine sample questions.

#### Interactive session using ipywidgets

The following utility class allows us to interact with the AI Assistant in a more natural way. We write out the question in an input box, and get the Assistant's answer. We can then continue our conversation.

In [52]:
grounding_chat = ChatComponent(llm=grounding_llm)
grounding_chat.render()

Box(children=(Image(value=b'GIF89a\xc8\x00\xc8\x00\xf7\x00\x00;Ch\x83\x90\xb7\xcf\xdc\xe8\xda\xec\xf1\xf1\xf2\…

In [22]:
history = [{"role": "user", "content": hallucination_example_prompt}]
bot_message = await grounding_llm.generate_async(messages=history)
print(bot_message['content'])

Parameter temperature does not exist for Bedrock
Parameter temperature does not exist for Bedrock
Hallucination rail can only be used with OpenAI LLM engines.Current LLM engine is NoneType.


<response>
- This Purina Pro Plan LiveClear Dry Cat Food for Kittens Chicken & Rice Formula is suitable for your active male cat weighing 2.26kg as it is a dry food suitable for cats of all breeds and life stages.
- As a cat food specially formulated for allergen management, this food is a good choice for your cat as it features ingredients derived from single protein sources, in this case chicken, to help support cats with food sensitivities and allergies. This makes it a hypoallergenic option.
- The primary protein of chicken is digestible and a quality source of protein to support your active cat's muscle growth and development while staying within an ideal weight.
- The dry kibble texture is convenient for both dry food feeders as well as cats that enjoy a mix of textures, as it can be easily served on its own or combined with wet foods.
- The allergen-managed formula is nutritionally complete to meet all your cat's dietary needs for a healthy immune system as he remains active ind